In [1]:
from qdev_wrappers.customised_instruments.composite_instruments.heterodyne_source import SimulatedHeterodyneSource, OneSourceHeterodyneSource, TwoSourceHeterodyneSource
from qdev_wrappers.customised_instruments.interfaces.microwave_source_interface import SimulatedMicrowaveSourceInterface

In [2]:
def print_params(instr):
    print(instr.name.upper())
    for k, v in instr.parameters.items():
        if k != 'IDN':
            print(k, ': ', v())
    print('\n')

### Simulated heterodyne source

In [3]:
simulated_source = SimulatedHeterodyneSource('heterodyne_source')

In [4]:
simulated_source.frequency(6e9)
simulated_source.power(-10)
simulated_source.demodulation_frequency(15e9)
simulated_source.localos_power(15)
simulated_source.status(1)
simulated_source.mode('basic')
print_params(simulated_source)

HETERODYNE_SOURCE
frequency :  6000000000.0
power :  -10
demodulation_frequency :  15000000000.0
localos_power :  15
status :  1
mode :  basic




### Single source heterodyne source

This should behave the same as the simulated heterodyne source, the only difference being that it is based on using one microwave source (which we simulate here). As a result the localos power is not settable but can be recorded at intiialisation as it is a harware parameter and the demodulation_frequency will likewise be fixed to zero.

In [5]:
microwave_source = SimulatedMicrowaveSourceInterface('microwave_source')
simulated_single_het_source = OneSourceHeterodyneSource('heterodyne_source_single', microwave_source, localos_power=-20)

TypeError: ("int() argument must be a string, a bytes-like object or a number, not 'NoneType'", 'getting microwave_source_pulsemod_state')

In [6]:
simulated_single_het_source.frequency(6e9)
simulated_single_het_source.power(-15)
simulated_single_het_source.status(1)
simulated_single_het_source.mode('sidebanded')
print_params(simulated_single_het_source)
print_params(microwave_source)

HETERODYNE_SOURCE_SINGLE
frequency :  6000000000.0
power :  -15
demodulation_frequency :  0
localos_power :  -20
status :  True
mode :  sidebanded


MICROWAVE_SOURCE
frequency :  6000000000.0
power :  -15
status :  True
pulsemod_state :  False
IQ_state :  True




The mode parameter controls the IQ state and pulsemod state of the cavity source. It has three options:<br>
- basic: IQ state off, pulsemod state off
- sidebanded: IQ state on, pulsemod state off
- sidebanded_modulated: IQ state on, pulsemod state on

In [7]:
simulated_single_het_source.mode('basic')
print(microwave_source.IQ_state())
print(microwave_source.pulsemod_state())

False
False


In [8]:
simulated_single_het_source.mode('sidebanded')
print(microwave_source.IQ_state())
print(microwave_source.pulsemod_state())

True
False


In [9]:
simulated_single_het_source.mode('sidebanded_modulated')
print(microwave_source.IQ_state())
print(microwave_source.pulsemod_state())

True
True


### Two source heterodyne source

Should have same functionality except that the second tone is provided by a separate microwave source so the demodulation frequency and power can be set dynamically. The 'basic' mode will also turn off the localos source status whereas for the other two it will match the value of the status parameter.

In [10]:
carrier_source = SimulatedMicrowaveSourceInterface('carrier_microwave_source')
localos_source = SimulatedMicrowaveSourceInterface('localos_microwave_source', IQ_option=False)
simulated_two_het_source = TwoSourceHeterodyneSource('heterodyne_source_two', carrier_source, localos_source)

In [11]:
simulated_two_het_source.frequency(6e9)
simulated_two_het_source.power(-15)
simulated_two_het_source.status(1)
simulated_two_het_source.demodulation_frequency(20e6)
simulated_two_het_source.localos_power(30)
simulated_two_het_source.mode('sidebanded_modulated')
print_params(simulated_two_het_source)
print_params(carrier_source)
print_params(localos_source)

HETERODYNE_SOURCE_TWO
frequency :  6000000000.0
power :  -15
demodulation_frequency :  20000000.0
localos_power :  30
status :  True
mode :  sidebanded_modulated


CARRIER_MICROWAVE_SOURCE
frequency :  6000000000.0
power :  -15
status :  True
pulsemod_state :  True
IQ_state :  True


LOCALOS_MICROWAVE_SOURCE
frequency :  6020000000.0
power :  30
status :  True
pulsemod_state :  False
IQ_state :  False




Mode acts as before excepth that in 'basic' the localos is also switched off (in other modes it's status matches the status parameter value)

In [12]:
simulated_two_het_source.mode('basic')
print(simulated_two_het_source.status())
print(localos_source.status())

True
False
